In [0]:
import numpy as np 
import pandas as pd

In [0]:
!curl -L -o train.csv https://drive.google.com/uc?id=1qkQphSpb7CjBYtd_93wWgISfo2H3HOcT
!curl -L -o test.csv https://drive.google.com/uc?id=1KstvnYRkkuBRI5AFE4Q-xeAVFmkp0haw

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   377    0   377    0     0    148      0 --:--:--  0:00:02 --:--:--   148
100 72.7M    0 72.7M    0     0  17.7M      0 --:--:--  0:00:04 --:--:-- 53.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   377    0   377    0     0    325      0 --:--:--  0:00:01 --:--:--   325
100 21.9M    0 21.9M    0     0  11.1M      0 --:--:--  0:00:01 --:--:-- 36.3M


In [0]:
!curl -L -o cc.ru.300.vec.gz https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1245M  100 1245M    0     0  12.1M      0  0:01:42  0:01:42 --:--:-- 12.3M


In [0]:
!gunzip cc.ru.300.vec.gz

gzip: cc.ru.300.vec already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [0]:
!chmod +x cc.ru.300.vec

In [0]:
!ls

cc.ru.300.vec	  sample_data	    submission8.csv  test.csv
cc.ru.300.vec.gz  submission10.csv  submission9.csv  train.csv


In [0]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
train.head()

,main_category,sub_category,question
0,4,32,Законно ли изменение количества дней листа нет...
1,1,8,"Погадайте что будут, у меня предчувствия?) К ч..."
2,10,66,Какой дровокол надёжней?
3,6,45,Зачем в старой Японии женщины бинтовали грудь ...
4,6,47,"Положив руку.. на ...ну что там у вас, мужчин"


In [0]:
len(train)

666666

In [0]:
train.main_category.value_counts()

20    42737
4     38047
3     37550
10    33813
16    31015
0     29520
13    28071
6     27146
19    26563
2     26455
18    25388
14    25266
12    23407
5     23394
7     23150
1     23051
27    22921
17    22215
24    22138
8     19958
22    18965
26    16226
11    15771
15    15491
25    15488
21    11846
9     11413
23     9661
Name: main_category, dtype: int64

In [0]:
import torch
from torch.utils.data import Dataset, DataLoader

from nltk.tokenize import word_tokenize, wordpunct_tokenize
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [0]:
def process_text(text):
    
    words = wordpunct_tokenize(text.lower())
    
    return words

In [0]:
word2freq = {}

for question in tqdm(train.question):
    
    words = process_text(question)
    
    for word in words:
        
        if word in word2freq:
            word2freq[word] += 1
        else:
            word2freq[word] = 1

100%|██████████| 666666/666666 [00:06<00:00, 106642.30it/s]


In [0]:
word2index = {'PAD': 0}
vectors = []
    
word2vec_file = open('cc.ru.300.vec')
    
n_words, embedding_dim = word2vec_file.readline().split()
n_words, embedding_dim = int(n_words), int(embedding_dim)

# Zero vector for PAD
vectors.append(np.zeros((1, embedding_dim)))

progress_bar = tqdm(desc='Read word2vec', total=n_words)

while True:

    line = word2vec_file.readline().strip()

    if not line:
        break
        
    current_parts = line.split()

    current_word = ' '.join(current_parts[:-embedding_dim])

    if current_word in word2freq:

        word2index[current_word] = len(word2index)

        current_vectors = current_parts[-embedding_dim:]
        current_vectors = np.array(list(map(float, current_vectors)))
        current_vectors = np.expand_dims(current_vectors, 0)

        vectors.append(current_vectors)

    progress_bar.update(1)

progress_bar.close()

word2vec_file.close()

vectors = np.concatenate(vectors)

Read word2vec: 100%|██████████| 2000000/2000000 [02:00<00:00, 16604.90it/s]


In [0]:
unk_words = [word for word in word2freq if word not in word2index]
unk_counts = [word2freq[word] for word in unk_words]
n_unk = sum(unk_counts) * 100 / sum(list(word2freq.values()))

sub_sample_unk_words = {word: word2freq[word] for word in unk_words}
sorted_unk_words = list(sorted(sub_sample_unk_words, key=lambda x: sub_sample_unk_words[x], reverse=True))

print('Мы не знаем {:.2f} % слов в датасете'.format(n_unk))
print('Количество неизвестных слов {} из {}, то есть {:.2f} % уникальных слов в словаре'.format(
    len(unk_words), len(word2freq), len(unk_words) * 100 / len(word2freq)))
print('В среднем каждое встречается {:.2f} раз'.format(np.mean(unk_counts)))
print()
print('Топ 5 невошедших слов:')

for i in range(5):
    print(sorted_unk_words[i], 'с количеством вхождениий -', word2freq[sorted_unk_words[i]])

Мы не знаем 3.02 % слов в датасете
Количество неизвестных слов 107889 из 328896, то есть 32.80 % уникальных слов в словаре
В среднем каждое встречается 2.08 раз

Топ 5 невошедших слов:
??? с количеством вхождениий - 8604
!!! с количеством вхождениий - 6976
?) с количеством вхождениий - 6613
?? с количеством вхождениий - 6327
"? с количеством вхождениий - 4581


In [0]:
class WordData(Dataset):
    
    def __init__(self, x_data, y_data, z_data, word2index, sequence_length=64, pad_token='PAD', verbose=True):
        
        super().__init__()
        
        self.x_data = []
        self.y_data = y_data
        self.z_data = z_data
        
        self.word2index = word2index
        self.sequence_length = sequence_length
        
        self.pad_token = pad_token
        self.pad_index = self.word2index[self.pad_token]
        
        self.load(x_data, verbose=verbose)
        
    @staticmethod
    def process_text(text):
    
        words = wordpunct_tokenize(text.lower())

        return words
        
    def load(self, data, verbose=True):
        
        data_iterator = tqdm(data, desc='Loading data', disable=not verbose)
        
        for text in data_iterator:
            words = self.process_text(text)
            indexed_words = self.indexing(words)
            self.x_data.append(indexed_words)
    
    def indexing(self, tokenized_text):

        return [self.word2index[token] for token in tokenized_text if token in self.word2index ]
    
    def padding(self, sequence):

        if len(sequence) > self.sequence_length:
            sequence = sequence[:self.sequence_length]
        elif len(sequence) < self.sequence_length:
            sequence = sequence + [self.pad_index] * (self.sequence_length - len(sequence))

        return sequence
    
    def __len__(self):
        
        return len(self.x_data)
    
    def __getitem__(self, idx):
        
        x = self.x_data[idx]
        x = self.padding(x)
        x = torch.Tensor(x).long()
        
        y = self.y_data[idx]
        z = self.z_data[idx]
        
        return x, y, z

In [0]:
x_train, x_validation, y_train, y_validation = train_test_split(train.question, train.main_category, test_size=0.1)
z_train = train.sub_category[y_train.index]
z_validation = train.sub_category[y_validation.index]

train_dataset = WordData(list(x_train), list(y_train), list(z_train), word2index)
train_loader = DataLoader(train_dataset, batch_size=64)

validation_dataset = WordData(list(x_validation), list(y_validation), list(z_validation), word2index)
validation_loader = DataLoader(validation_dataset, batch_size=64)

test_dataset = WordData(list(test.question), np.zeros((test.shape[0])), np.zeros((test.shape[0])), word2index)
test_loader = DataLoader(test_dataset, batch_size=64)

Loading data: 100%|██████████| 200000/200000 [00:02<00:00, 88280.85it/s]


In [0]:
for x, y, z in test_loader:
    break

In [0]:
x.shape

torch.Size([64, 64])

In [0]:
y

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [0]:
z

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [0]:
main_classes = train.main_category.unique().shape[0]
sub_classes = train.sub_category.unique().shape[0]

In [0]:
import torch.nn.functional as F

In [0]:
class DeepAverageNetwork(torch.nn.Module):
    
    def __init__(self, embedding_matrix, n_classes_main, n_classes_sub):
        
        super().__init__()
        

        self.filter_sizes = [1,2,3,5,6,7,8,9,10]
        self.num_filters = 512
        self.embedding_layer = torch.nn.Embedding.from_pretrained(torch.Tensor(embedding_matrix))
        self.lstm = torch.nn.LSTM(300, 128, num_layers=2, batch_first=True, bidirectional=True)
        self.convs1 = torch.nn.ModuleList([torch.nn.Conv1d(256, self.num_filters, K) for K in self.filter_sizes])
        self.convs2 = torch.nn.ModuleList([torch.nn.Conv1d(512, self.num_filters, K) for K in self.filter_sizes[:4]])
        self.layers = torch.nn.Sequential(torch.nn.ReLU(),
                                  torch.nn.Dropout(0.2),
                                  torch.nn.Linear(256, 128),
                                  torch.nn.ReLU(),
                                  torch.nn.Dropout(0.2))
        self.layers2 = torch.nn.Sequential(torch.nn.ReLU(),
                                  torch.nn.Dropout(0.2),
                                  torch.nn.Linear(self.num_filters*len(self.filter_sizes), 128),
                                  torch.nn.ReLU(),
                                  torch.nn.Dropout(0.2))
        self.linear_main = torch.nn.Linear(128, n_classes_main)
        self.linear_sub = torch.nn.Linear(128, n_classes_sub)


    def forward(self, x):
        
        sequence_lengths = (x > 0).sum(dim=1)
        sequence_lengths[sequence_lengths == 0.] = 1
        
        x = self.embedding_layer(x)
        
        x, hidden = self.lstm(x)

        x= [F.relu(conv(x.transpose(1,2))) for conv in self.convs1]

        
        x= [F.max_pool1d(i, i.size(2)).squeeze() for i in x] 
        x = torch.cat(x, 1)

        x = self.layers2(x)
        x1 = self.linear_main(x)
        x2 = self.linear_sub(x)
        
        return x1, x2

In [0]:
model = DeepAverageNetwork(embedding_matrix=vectors, n_classes_main=main_classes, n_classes_sub=sub_classes)

Epoch 1:  95%|█████████▍| 568704/599999 [08:10<00:25, 1222.84it/s, train_loss=2.29]

In [0]:
with torch.no_grad():
    pred1, pred2 = model(x)

In [0]:
pred1.shape

torch.Size([64, 28])

In [0]:
pred2.shape

torch.Size([64, 209])

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [0]:
device

device(type='cuda')

In [0]:
# let's make a weighted training

def get_weights(label):
        lab2id = lab2id = {x: i for i, x in enumerate(sorted(set(train[label][i].item() for i in range(train[label].shape[0]))))}
        weights = torch.FloatTensor([
                    1.0 / (train[label] == i).sum().item()
                    for i in range(len(lab2id))
                ]).to(device)
        weights = weights / weights.sum()
        return weights

In [0]:
import torch.nn.functional as F

In [0]:
# criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters())

model = model.to(device)
# criterion = criterion.to(device)

In [0]:
weights_main = get_weights('main_category')
weights_sub = get_weights('sub_category')

In [0]:
epochs = 5
losses = []
best_test_loss = 10.

test_f1 = []

for n_epoch in range(epochs):
    
    train_losses = []
    test_losses = []
    test_targets = []
    test_pred_class = []
    early_stoping_value = 0
    
    progress_bar = tqdm(total=len(train_loader.dataset), desc='Epoch {}'.format(n_epoch + 1))
    
    model.train()
    
    for x, y, z in train_loader:

        x = x.to(device)
        y = y.to(device)
        z = z.to(device)
        
        optimizer.zero_grad()
        
        pred1, pred2 = model(x)
        loss1 = F.cross_entropy(pred1, y, weight=weights_main) #criterion(pred, y)
        loss2 = F.cross_entropy(pred2, z, weight=weights_sub)
        
        loss = (loss1 + loss2)/2
        
        loss.backward()
        
        optimizer.step()
        
        train_losses.append(loss.item())
        losses.append(loss.item())
        
        progress_bar.set_postfix(train_loss = np.mean(losses[-500:]))

        progress_bar.update(x.shape[0])
        
    progress_bar.close()
    
    model.eval()
    
    for x, y, z in validation_loader:
        
        x = x.to(device)
        y = y.to(device)
        z = z.to(device)

        with torch.no_grad():

            pred1, pred2 = model(x)

            pred1 = pred1.cpu()
            pred2 = pred2.cpu()
            y = y.cpu()
            z = z.cpu()

            test_targets.append(y.numpy())
            test_pred_class.append(np.argmax(pred1, axis=1))

            loss = F.cross_entropy(pred1, y) #criterion(pred, y)

            test_losses.append(loss.item())
        
    mean_test_loss = np.mean(test_losses)

    test_targets = np.concatenate(test_targets).squeeze()
    test_pred_class = np.concatenate(test_pred_class).squeeze()

    f1 = f1_score(test_targets, test_pred_class, average='micro')

    test_f1.append(f1)
    
    print()
    print('Losses: train - {:.3f}, test - {:.3f}'.format(np.mean(train_losses), mean_test_loss))

    print('F1 test - {:.3f}'.format(f1))
    
    
        
    # Early stopping:
    if mean_test_loss < best_test_loss:
        best_test_loss = mean_test_loss
        early_stoping_value=0
    else:
        early_stoping_value+=1
        if early_stoping_value>=2:
            print('Early stopping')
            break

In [0]:
y

In [0]:
model.eval()

predictions = []

for x, _, __ in test_loader:

    x = x.to(device)

    with torch.no_grad():

        pred1, pred2 = model(x)

        pred1 = pred1.cpu()
        
        predictions.append(np.argmax(pred1, axis=1))
        
predictions = np.concatenate(predictions).squeeze()

In [0]:
test.head()

In [0]:
test['main_category'] = predictions

In [0]:
test = test[['index', 'main_category']]

In [0]:
test.head()

In [0]:
test.to_csv('submission11.csv', index=False)

In [0]:
from google.colab import files
files.download('submission11.csv') 